In [ ]:
from google.colab import files

# Wgrywanie plików
uploaded = files.upload()


Saving dataset.test.txt to dataset.test.txt
Saving dataset.train.txt to dataset.train.txt
Saving Zmienne.txt to Zmienne.txt


In [ ]:
import pandas as pd

# Załadowanie nazw zmiennych
with open('Zmienne.txt', 'r') as f:
    column_names = f.read().splitlines()

# Załadowanie danych
train_data = pd.read_csv('dataset.train.txt', delim_whitespace=True, header=None, names=column_names)
test_data = pd.read_csv('dataset.test.txt', delim_whitespace=True, header=None, names=column_names)

# Podgląd danych
print("Train Data:")
print(train_data.head())

print("\nTest Data:")
print(test_data.head())


<ipython-input-2-2299c07f1ce7>:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  train_data = pd.read_csv('dataset.train.txt', delim_whitespace=True, header=None, names=column_names)
<ipython-input-2-2299c07f1ce7>:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  test_data = pd.read_csv('dataset.test.txt', delim_whitespace=True, header=None, names=column_names)


Train Data:
   unit_number  time_in_cycles  op_setting_1  op_setting_2  op_setting_3  \
0            1               1       -0.0007       -0.0004         100.0   
1            1               2        0.0019       -0.0003         100.0   
2            1               3       -0.0043        0.0003         100.0   
3            1               4        0.0007        0.0000         100.0   
4            1               5       -0.0019       -0.0002         100.0   

   sensor_measurement_1  sensor_measurement_2  sensor_measurement_3  \
0                518.67                641.82               1589.70   
1                518.67                642.15               1591.82   
2                518.67                642.35               1587.99   
3                518.67                642.35               1582.79   
4                518.67                642.37               1582.85   

   sensor_measurement_4  sensor_measurement_5  ...  sensor_measurement_12  \
0               1400.60    

In [ ]:
# Sprawdzenie braków w danych
print("\nMissing values in training data:")
print(train_data.isnull().sum())

print("\nMissing values in testing data:")
print(test_data.isnull().sum())

# Statystyki opisowe
print("\nDescriptive statistics of training data:")
print(train_data.describe())

# Sprawdzenie korelacji
correlation_matrix = train_data.corr()
print("\nCorrelation with target variable (time_in_cycles):")
print(correlation_matrix['time_in_cycles'].sort_values(ascending=False))



Missing values in training data:
unit_number              0
time_in_cycles           0
op_setting_1             0
op_setting_2             0
op_setting_3             0
sensor_measurement_1     0
sensor_measurement_2     0
sensor_measurement_3     0
sensor_measurement_4     0
sensor_measurement_5     0
sensor_measurement_6     0
sensor_measurement_7     0
sensor_measurement_8     0
sensor_measurement_9     0
sensor_measurement_10    0
sensor_measurement_11    0
sensor_measurement_12    0
sensor_measurement_13    0
sensor_measurement_14    0
sensor_measurement_15    0
sensor_measurement_16    0
sensor_measurement_17    0
sensor_measurement_18    0
sensor_measurement_19    0
sensor_measurement_20    0
sensor_measurement_21    0
dtype: int64

Missing values in testing data:
unit_number              0
time_in_cycles           0
op_setting_1             0
op_setting_2             0
op_setting_3             0
sensor_measurement_1     0
sensor_measurement_2     0
sensor_measurement_3     0
se

In [ ]:
# Lista zmiennych do odrzucenia na podstawie analizy
drop_columns = [
    'op_setting_3', 'sensor_measurement_1', 'sensor_measurement_5',
    'sensor_measurement_10', 'sensor_measurement_16', 'sensor_measurement_18',
    'sensor_measurement_19', 'op_setting_1', 'op_setting_2',
    'sensor_measurement_20', 'sensor_measurement_21',
    'sensor_measurement_7', 'sensor_measurement_12'
]

# Usunięcie nieprzydatnych zmiennych
filtered_train_data = train_data.drop(columns=drop_columns)
filtered_test_data = test_data.drop(columns=drop_columns)

# Podgląd przetworzonych danych
print("\nPrzefiltrowane dane treningowe (pierwsze 5 wierszy):")
print(filtered_train_data.head())

print("\nPrzefiltrowane dane testowe (pierwsze 5 wierszy):")
print(filtered_test_data.head())



Przefiltrowane dane treningowe (pierwsze 5 wierszy):
   unit_number  time_in_cycles  sensor_measurement_2  sensor_measurement_3  \
0            1               1                641.82               1589.70   
1            1               2                642.15               1591.82   
2            1               3                642.35               1587.99   
3            1               4                642.35               1582.79   
4            1               5                642.37               1582.85   

   sensor_measurement_4  sensor_measurement_6  sensor_measurement_8  \
0               1400.60                 21.61               2388.06   
1               1403.14                 21.61               2388.04   
2               1404.20                 21.61               2388.08   
3               1401.87                 21.61               2388.11   
4               1406.22                 21.61               2388.06   

   sensor_measurement_9  sensor_measurement_11  se

Odrzucamy zmienne o niskiej lub negatywnej korelacji op_setting_1, op_setting_2, sensor_measurement_20, sensor_measurement_21, sensor_measurement_7, sensor_measurement_12.

Zmienne które mają zerowe odchylenie standardowe (std = 0). Oznacza to, że te zmienne są stałe i nie wnoszą informacji, więc należy je odrzucić. op_setting_3, sensor_measurement_1, sensor_measurement_5, sensor_measurement_10, sensor_measurement_16, sensor_measurement_18, i sensor_measurement_19


In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import r2_score

# Wybór zmiennych objaśniających
features = ['sensor_measurement_11', 'sensor_measurement_4', 'sensor_measurement_15',
            'sensor_measurement_17', 'sensor_measurement_2', 'sensor_measurement_3']
target = 'time_in_cycles'

# Przygotowanie danych treningowych
X_train = filtered_train_data[features]
y_train = filtered_train_data[target]

# Dodanie stałej do modelu
X_train = sm.add_constant(X_train)

# Budowa modelu regresji liniowej
model = sm.OLS(y_train, X_train).fit()

# Wyświetlenie podsumowania modelu
print(model.summary())

# Przygotowanie danych testowych
X_test = filtered_test_data[features]
y_test = filtered_test_data[target]

# Dodanie stałej do danych testowych
X_test = sm.add_constant(X_test)

# Przewidywanie na danych testowych
y_pred = model.predict(X_test)

# Obliczenie współczynnika determinacji R² dla danych testowych
r2_test = r2_score(y_test, y_pred)
print(f"\nWspółczynnik determinacji (R²) dla danych testowych: {r2_test:.2f}")


                            OLS Regression Results                            
Dep. Variable:         time_in_cycles   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.460
Method:                 Least Squares   F-statistic:                     2934.
Date:                Tue, 28 Jan 2025   Prob (F-statistic):               0.00
Time:                        17:48:28   Log-Likelihood:            -1.1023e+05
No. Observations:               20631   AIC:                         2.205e+05
Df Residuals:                   20624   BIC:                         2.205e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -1.583e+

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Podział danych treningowych na zbiór treningowy i walidacyjny
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_reduced, y_train, test_size=0.2, random_state=42
)

# Siatka hyperparametrów dla alfa
alphas = [0.1, 1, 10, 100, 1000]

# Ridge Regression
ridge = Ridge()
ridge_params = {'alpha': alphas}
ridge_search = GridSearchCV(ridge, ridge_params, cv=5, scoring='r2')
ridge_search.fit(X_train_split, y_train_split)

# Lasso Regression
lasso = Lasso(max_iter=10000)
lasso_params = {'alpha': alphas}
lasso_search = GridSearchCV(lasso, lasso_params, cv=5, scoring='r2')
lasso_search.fit(X_train_split, y_train_split)

# Elastic Net Regression
elastic_net = ElasticNet(max_iter=10000)
elastic_net_params = {'alpha': alphas, 'l1_ratio': [0.1, 0.5, 0.9]}  # Kombinacja L1 i L2
elastic_net_search = GridSearchCV(elastic_net, elastic_net_params, cv=5, scoring='r2')
elastic_net_search.fit(X_train_split, y_train_split)

# Najlepsze wartości alpha i l1_ratio
print("\nNajlepszy model Ridge:")
print(f"Alpha: {ridge_search.best_params_['alpha']}, R^2: {ridge_search.best_score_:.2f}")

print("\nNajlepszy model Lasso:")
print(f"Alpha: {lasso_search.best_params_['alpha']}, R^2: {lasso_search.best_score_:.2f}")

print("\nNajlepszy model Elastic Net:")
print(f"Alpha: {elastic_net_search.best_params_['alpha']}, L1_ratio: {elastic_net_search.best_params_['l1_ratio']}, R^2: {elastic_net_search.best_score_:.2f}")

# Ocena na zbiorze walidacyjnym
ridge_best = ridge_search.best_estimator_
lasso_best = lasso_search.best_estimator_
elastic_net_best = elastic_net_search.best_estimator_

ridge_pred = ridge_best.predict(X_val_split)
lasso_pred = lasso_best.predict(X_val_split)
elastic_net_pred = elastic_net_best.predict(X_val_split)

print("\nOcena modeli na zbiorze walidacyjnym:")
print(f"Ridge Regression: R^2 = {r2_score(y_val_split, ridge_pred):.2f}, MSE = {mean_squared_error(y_val_split, ridge_pred):.2f}")
print(f"Lasso Regression: R^2 = {r2_score(y_val_split, lasso_pred):.2f}, MSE = {mean_squared_error(y_val_split, lasso_pred):.2f}")
print(f"Elastic Net Regression: R^2 = {r2_score(y_val_split, elastic_net_pred):.2f}, MSE = {mean_squared_error(y_val_split, elastic_net_pred):.2f}")



Najlepszy model Ridge:
Alpha: 0.1, R^2: 0.45

Najlepszy model Lasso:
Alpha: 0.1, R^2: 0.45

Najlepszy model Elastic Net:
Alpha: 0.1, L1_ratio: 0.9, R^2: 0.44

Ocena modeli na zbiorze walidacyjnym:
Ridge Regression: R^2 = 0.45, MSE = 2600.40
Lasso Regression: R^2 = 0.44, MSE = 2635.28
Elastic Net Regression: R^2 = 0.43, MSE = 2671.46


In [ ]:
import pandas as pd

# Pobranie współczynników dla każdego modelu
lm_coeffs = pd.Series(ridge_best.coef_, index=X_train_reduced.columns)
ridge_coeffs = pd.Series(ridge_best.coef_, index=X_train_reduced.columns)
lasso_coeffs = pd.Series(lasso_best.coef_, index=X_train_reduced.columns)
elastic_net_coeffs = pd.Series(elastic_net_best.coef_, index=X_train_reduced.columns)

# Tworzenie DataFrame ze współczynnikami
coefficients_df = pd.DataFrame({
    "OLS": lm_coeffs,
    "Ridge": ridge_coeffs,
    "Lasso": lasso_coeffs,
    "Elastic Net": elastic_net_coeffs
})

# Wyświetlenie współczynników
print("\nWspółczynniki dla każdego modelu:")
print(coefficients_df)



Współczynniki dla każdego modelu:
                              OLS       Ridge       Lasso  Elastic Net
const                    0.000000    0.000000    0.000000     0.000000
sensor_measurement_4     2.163532    2.163532    2.543251     2.762000
sensor_measurement_15  298.991978  298.991978  112.899620     6.606272
sensor_measurement_17    7.634122    7.634122    8.550155     9.047812
sensor_measurement_3     1.626615    1.626615    1.835230     1.957685


In [ ]:
from sklearn.metrics import r2_score

# Obliczenie R^2 dla danych treningowych
r2_ridge_train = ridge_best.score(X_train_reduced, y_train)
r2_lasso_train = lasso_best.score(X_train_reduced, y_train)
r2_net_train = elastic_net_best.score(X_train_reduced, y_train)

# Obliczenie R^2 dla danych testowych
r2_ridge_test = r2_score(y_test, ridge_best.predict(X_test_reduced))
r2_lasso_test = r2_score(y_test, lasso_best.predict(X_test_reduced))
r2_net_test = r2_score(y_test, elastic_net_best.predict(X_test_reduced))

# Tworzenie ramki danych
r2_df = pd.DataFrame({
    "OLS": ["0.461", "0.21"],
    "Ridge": [r2_ridge_train, r2_ridge_test],
    "Lasso": [r2_lasso_train, r2_lasso_test],
    "Elastic Net": [r2_net_train, r2_net_test]
}, index=["training", "testing"])

# Wyświetlenie wyników
print("\nTabela współczynnika determinacji (R^2) dla modeli:")
print(r2_df)



Tabela współczynnika determinacji (R^2) dla modeli:
            OLS     Ridge     Lasso  Elastic Net
training  0.461  0.449097  0.444458     0.438379
testing    0.21  0.188144  0.179914     0.169580


OLS (0.461 → 0.21)
Przeuczenie (overfitting) – model dobrze dopasował się do danych treningowych, ale nie generalizuje dobrze na nowych danych.
Zbyt duża różnica między R2  dla treningu i testu wskazuje na dopasowanie do szumu w danych.

Ridge, Lasso, Elastic Net (~0.44 → ~0.18)
Lepsza stabilność niż OLS, ale nadal niski R2 na testach.
Możliwe niedopasowanie (underfitting) – zbyt silna regularyzacja może tłumić ważne cechy.
Elastic Net ma najniższy wynik, co oznacza, że regularyzacja L1+L2 jest zbyt agresywna.